Com a coluna conteudo resumida

In [2]:
import pandas as pd
df_final = pd.read_csv("../dados/df_final.csv")

Extrair informações importantes de texto

Limite de tokens: Gemini 1.5 Flash suporta até 1 milhão de tokens contextuais, mas é mais rápido e mais barato com textos curtos. Se o documento for muito longo, divida em trechos.

Prompt estruturado = melhor resultado: Use listas, marcadores e perguntas diretas.

Padronize o input: Remova cabeçalhos, assinaturas ou trechos irrelevantes.

Verifique consistência: LLMs podem gerar variações. Se precisar de formato JSON, peça explicitamente.

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Carrega variáveis do .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

def extrair_info(texto):
    prompt = f"""
    Extraia as seguintes informações do texto abaixo e retorne em formato JSON:

    {{
      "tipo_decisao": "...",
      "condenacao": true,
      "valores_multas": ["..."],
      "seguiu_nota_tecnica": true
    }}

    Texto:
    \"\"\"{texto}\"\"\"
    """
    response = model.generate_content(prompt)
    return response.text


ImportError: cannot import name 'InputOutputText' from 'vertexai.language_models' (/home/gabriel/.local/lib/python3.10/site-packages/vertexai/language_models/__init__.py)